Task 2: Lookalike Model

In [30]:
customer_features = mergedlist.groupby('CustomerID').agg({
    'TotalValue': 'sum',  
    'Quantity': 'sum',   
    'ProductID': lambda x: x.mode()[0]  
})


In [32]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

customer_transactions = transactions.groupby('CustomerID').agg(
    total_spend=('TotalValue', 'sum'),
    total_quantity=('Quantity', 'sum'),
    num_transactions=('TransactionID', 'count')
).reset_index()

customer_data = customers.merge(customer_transactions, on='CustomerID', how='left')
customer_data.fillna(0, inplace=True)

encoder = OneHotEncoder(sparse=False, drop='first')
region_encoded = encoder.fit_transform(customer_data[['Region']])

scaler = MinMaxScaler()
numerical_features = scaler.fit_transform(customer_data[['total_spend', 'total_quantity', 'num_transactions']])

customer_features = np.hstack([region_encoded, numerical_features])

similarity_matrix = cosine_similarity(customer_features)


In [ ]:
import numpy as np
import pandas as pd

customer_ids = customer_data['CustomerID'].tolist()

similarity_matrix = cosine_similarity(customer_features)

recommendations = {}
for i, customer in enumerate(customer_ids):
    similarities = list(enumerate(similarity_matrix[i]))
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    sorted_similarities = [(customer_ids[j], score) for j, score in sorted_similarities if j != i]
    recommendations[customer] = sorted_similarities[:3]

recommendations_df = pd.DataFrame([
    {'Customer': customer, 'Top Recommendations': recs}
    for customer, recs in recommendations.items()
])
print(recommendations_df)


    Customer                                Top Recommendations
0      C0001  [(C0107, 0.9997970719930916), (C0137, 0.999692...
1      C0002  [(C0084, 0.9979443840440271), (C0142, 0.997881...
2      C0003  [(C0190, 0.997374997052909), (C0133, 0.9969677...
3      C0004  [(C0113, 0.9994191453118594), (C0102, 0.997993...
4      C0005  [(C0178, 0.9998657137098317), (C0101, 0.999554...
..       ...                                                ...
195    C0196  [(C0170, 0.9956741491873117), (C0037, 0.994704...
196    C0197  [(C0132, 0.999595815958913), (C0073, 0.9995753...
197    C0198  [(C0063, 0.9974044889548139), (C0009, 0.996313...
198    C0199  [(C0166, 0.9980773121890072), (C0197, 0.996891...
199    C0200  [(C0143, 0.9992241537956438), (C0028, 0.999041...

[200 rows x 2 columns]


In [24]:
lookalikes = pd.DataFrame([
    {'Customer': customer, 'Top Recommendations': recs}
    for customer, recs in recommendations.items()
])


In [25]:
lookalikes['Top Recommendations'] = lookalikes['Top Recommendations'].apply(
    lambda recs: ', '.join([f"{customer} ({score:.2f})" for customer, score in recs])
)


In [26]:
lookalikes.to_csv('C:/Users/DELL/Downloads/Telegram Desktop/Zeotap/Koya_Madhusudhana_Rao_Lookalike.csv', index=False)
